# Fast tokenizers in the QA pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9802603125572205,
 'start': 78,
 'end': 106,
 'answer': 'Jax, PyTorch, and TensorFlow'}

In [3]:
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question_answerer(question=question, context=long_context)

{'score': 0.9714871048927307,
 'start': 1892,
 'end': 1919,
 'answer': 'Jax, PyTorch and TensorFlow'}

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

In [5]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([1, 67]) torch.Size([1, 67])


In [6]:
import torch

sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask1 = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask1[0] = False
mask = torch.tensor(mask1)[None]

start_logits[mask] = -10000  # Set to -10000 for each element with value True, i.e. the question part
end_logits[mask] = -10000

In [13]:
sequence_ids

[None,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 None,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 None]

In [12]:
mask

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True]])

In [14]:
start_logits

tensor([[-4.4952e+00, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04,
         -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04,
         -1.5920e+00, -1.0857e+00, -5.0981e+00, -2.9331e+00, -3.4070e+00,
          2.2467e+00,  5.1563e+00, -1.3602e+00, -2.2209e+00, -9.6861e-01,
         -4.8112e+00, -2.2527e+00,  1.4383e+00,  1.0121e+01, -1.5311e+00,
          2.2685e+00, -1.8951e+00, -2.2108e+00, -4.2142e+00, -2.5571e+00,
         -2.3252e+00, -2.6046e+00,  1.7047e+00, -1.9867e+00, -1.7211e+00,
         -5.4149e-01, -2.0239e+00, -4.4246e+00, -5.1012e+00, -4.4966e+00,
         -7.8940e+00, -6.7200e+00, -4.6759e+00, -6.3278e+00, -4.8339e+00,
         -5.1839e+00, -3.3724e+00, -7.4120e+00, -8.1542e+00, -4.4871e+00,
         -7.4659e+00, -4.3293e+00, -4.2293e+00, -3.1903e+00, -7.9467e+00,
         -5.2665e+00, -7.5902e+00, -5.0570e+00, -7.4476e+00, -7.9083e+00,
         -6.5951e+00, -7.4061e+00, -8.8821e+00, -7.6749e+00, -6.9879e+00,
         -7.0466e+00, -1.0000e+04]], g

In [15]:
end_logits

tensor([[-2.3958e+00, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04,
         -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04,
         -3.1757e+00, -1.1649e+00, -7.0748e+00, -5.2875e+00, -6.8611e+00,
         -5.1769e+00,  3.7892e+00, -4.4408e+00, -7.6687e-01, -3.9180e+00,
         -2.1634e+00,  1.8116e+00, -1.4678e+00,  2.0508e+00,  1.5404e-03,
         -1.5531e+00, -6.9470e-01, -1.3466e+00, -1.6879e+00,  4.0826e+00,
          1.1467e+00, -3.7881e-01,  6.0774e-01,  1.2281e+00,  5.8202e-01,
          1.0657e+01,  5.8794e+00, -5.7342e+00, -7.0719e+00, -6.8077e+00,
         -7.1513e+00, -5.3228e+00, -3.4305e+00, -4.2575e+00,  2.2268e+00,
         -4.1297e-01, -6.8944e+00, -7.9381e+00, -8.3298e+00, -5.6078e+00,
         -8.9589e+00, -5.5772e+00, -5.7309e+00, -1.9592e+00, -7.8078e+00,
         -2.3823e+00, -7.2457e+00, -6.1642e+00, -4.2830e+00, -8.0948e+00,
         -8.0364e+00, -4.5566e+00, -7.6585e+00, -7.3241e+00, -2.2402e+00,
         -1.8462e+00, -1.0000e+04]], g

In [16]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]

In [17]:
start_probabilities

tensor([4.4531e-07, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.1185e-06, 1.3470e-05,
        2.4368e-07, 2.1236e-06, 1.3220e-06, 3.7722e-04, 6.9219e-03, 1.0237e-05,
        4.3289e-06, 1.5143e-05, 3.2463e-07, 4.1933e-06, 1.6808e-04, 9.9179e-01,
        8.6288e-06, 3.8557e-04, 5.9956e-06, 4.3725e-06, 5.8977e-07, 3.0929e-06,
        3.8999e-06, 2.9493e-06, 2.1940e-04, 5.4713e-06, 7.1354e-06, 2.3212e-05,
        5.2711e-06, 4.7788e-07, 2.4291e-07, 4.4467e-07, 1.4879e-08, 4.8133e-08,
        3.7169e-07, 7.1242e-08, 3.1735e-07, 2.2365e-07, 1.3685e-06, 2.4093e-08,
        1.1470e-08, 4.4891e-07, 2.2828e-08, 5.2562e-07, 5.8092e-07, 1.6419e-06,
        1.4114e-08, 2.0591e-07, 2.0161e-08, 2.5390e-07, 2.3251e-08, 1.4667e-08,
        5.4533e-08, 2.4235e-08, 5.5390e-09, 1.8524e-08, 3.6818e-08, 3.4721e-08,
        0.0000e+00], grad_fn=<SelectBackward0>)

In [18]:
end_probabilities

tensor([2.1185e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.7129e-07, 7.2546e-06,
        1.9679e-08, 1.1754e-07, 2.4366e-08, 1.3130e-07, 1.0284e-03, 2.7411e-07,
        1.0801e-05, 4.6233e-07, 2.6730e-06, 1.4233e-04, 5.3586e-06, 1.8078e-04,
        2.3292e-05, 4.9207e-06, 1.1610e-05, 6.0494e-06, 4.3002e-06, 1.3790e-03,
        7.3201e-05, 1.5923e-05, 4.2704e-05, 7.9413e-05, 4.1620e-05, 9.8838e-01,
        8.3161e-03, 7.5197e-08, 1.9735e-08, 2.5704e-08, 1.8229e-08, 1.1346e-07,
        7.5278e-07, 3.2926e-07, 2.1558e-04, 1.5388e-05, 2.3568e-08, 8.2993e-09,
        5.6096e-09, 8.5331e-08, 2.9904e-09, 8.7980e-08, 7.5447e-08, 3.2784e-06,
        9.4549e-09, 2.1473e-06, 1.6586e-08, 4.8915e-08, 3.2096e-07, 7.0959e-09,
        7.5224e-09, 2.4413e-07, 1.0978e-08, 1.5336e-08, 2.4753e-06, 3.6704e-06,
        0.0000e+00], grad_fn=<SelectBackward0>)

In [19]:
scores = start_probabilities[:, None] * end_probabilities[None, :]

In [24]:
start_probabilities.shape

torch.Size([67])

In [25]:
start_probabilities[:, None]

tensor([[4.4531e-07],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [8.1185e-06],
        [1.3470e-05],
        [2.4368e-07],
        [2.1236e-06],
        [1.3220e-06],
        [3.7722e-04],
        [6.9219e-03],
        [1.0237e-05],
        [4.3289e-06],
        [1.5143e-05],
        [3.2463e-07],
        [4.1933e-06],
        [1.6808e-04],
        [9.9179e-01],
        [8.6288e-06],
        [3.8557e-04],
        [5.9956e-06],
        [4.3725e-06],
        [5.8977e-07],
        [3.0929e-06],
        [3.8999e-06],
        [2.9493e-06],
        [2.1940e-04],
        [5.4713e-06],
        [7.1354e-06],
        [2.3212e-05],
        [5.2711e-06],
        [4.7788e-07],
        [2.4291e-07],
        [4.4467e-07],
        [1.4879e-08],
        [4.8133e-08],
        [3.7169e-07],
        [7.1242e-08],
        [3.1735e-07],
        [2

In [26]:
end_probabilities[None, :]

tensor([[2.1185e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.7129e-07, 7.2546e-06,
         1.9679e-08, 1.1754e-07, 2.4366e-08, 1.3130e-07, 1.0284e-03, 2.7411e-07,
         1.0801e-05, 4.6233e-07, 2.6730e-06, 1.4233e-04, 5.3586e-06, 1.8078e-04,
         2.3292e-05, 4.9207e-06, 1.1610e-05, 6.0494e-06, 4.3002e-06, 1.3790e-03,
         7.3201e-05, 1.5923e-05, 4.2704e-05, 7.9413e-05, 4.1620e-05, 9.8838e-01,
         8.3161e-03, 7.5197e-08, 1.9735e-08, 2.5704e-08, 1.8229e-08, 1.1346e-07,
         7.5278e-07, 3.2926e-07, 2.1558e-04, 1.5388e-05, 2.3568e-08, 8.2993e-09,
         5.6096e-09, 8.5331e-08, 2.9904e-09, 8.7980e-08, 7.5447e-08, 3.2784e-06,
         9.4549e-09, 2.1473e-06, 1.6586e-08, 4.8915e-08, 3.2096e-07, 7.0959e-09,
         7.5224e-09, 2.4413e-07, 1.0978e-08, 1.5336e-08, 2.4753e-06, 3.6704e-06,
         0.0000e+00]], grad_fn=<SliceBackward0>)

In [23]:
scores.shape

torch.Size([67, 67])

In [27]:
scores = torch.triu(scores)

In [33]:
max_index = scores.argmax().item()
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
print(scores[start_index, end_index])

tensor(0.9803, grad_fn=<SelectBackward0>)


In [28]:
scores.argmax()

tensor(1576)

In [29]:
scores.argmax().item()

1576

In [32]:
scores.shape[1]

67

In [34]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

In [35]:
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
}
print(result)

{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 78, 'end': 106, 'score': tensor(0.9803, grad_fn=<SelectBackward0>)}


In [36]:
inputs = tokenizer(question, long_context)
print(len(inputs["input_ids"]))

461


In [37]:
inputs

{'input_ids': [101, 5979, 1996, 3776, 9818, 1171, 100, 25267, 136, 102, 100, 25267, 131, 1426, 1104, 1103, 2051, 21239, 2101, 100, 25267, 2790, 4674, 1104, 3073, 4487, 9044, 3584, 1106, 3870, 8249, 1113, 6685, 1216, 1112, 5393, 117, 1869, 16026, 117, 2304, 10937, 117, 7584, 7317, 2734, 117, 5179, 117, 3087, 3964, 1105, 1167, 1107, 1166, 1620, 3483, 119, 2098, 6457, 1110, 1106, 1294, 5910, 118, 2652, 21239, 2101, 5477, 1106, 1329, 1111, 2490, 119, 100, 25267, 2790, 20480, 1116, 1106, 1976, 9133, 1105, 1329, 1343, 3073, 4487, 9044, 3584, 1113, 170, 1549, 3087, 117, 2503, 118, 9253, 1172, 1113, 1240, 1319, 2233, 27948, 1105, 1173, 2934, 1172, 1114, 1103, 1661, 1113, 1412, 2235, 10960, 119, 1335, 1103, 1269, 1159, 117, 1296, 185, 25669, 8613, 13196, 13682, 1126, 4220, 1110, 3106, 2484, 20717, 1673, 1105, 1169, 1129, 5847, 1106, 9396, 3613, 1844, 7857, 119, 2009, 1431, 146, 1329, 11303, 1468, 136, 122, 119, 12167, 118, 1106, 118, 1329, 1352, 118, 1104, 118, 1103, 118, 1893, 3584, 131, 118, 

In [38]:
inputs = tokenizer(question, long_context, max_length=384, truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

[CLS] Which deep learning libraries back [UNK] Transformers? [SEP] [UNK] Transformers : State of the Art NLP [UNK] Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. Its aim is to make cutting - edge NLP easier to use for everyone. [UNK] Transformers provides APIs to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. Why should I use transformers? 1. Easy - to - use state - of - the - art models : - High performance on NLU and NLG tasks. - Low barrier to entry for educators and practitioners. - Few user - facing abstractions with just three classes to learn. - A unified A

In [39]:
sentence = "This sentence is not too long but we are going to split it anyway."
inputs = tokenizer(
    sentence, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] This sentence is not [SEP]
[CLS] is not too long [SEP]
[CLS] too long but we [SEP]
[CLS] but we are going [SEP]
[CLS] are going to split [SEP]
[CLS] to split it anyway [SEP]
[CLS] it anyway. [SEP]


In [40]:
inputs["input_ids"]

[[101, 1188, 5650, 1110, 1136, 102],
 [101, 1110, 1136, 1315, 1263, 102],
 [101, 1315, 1263, 1133, 1195, 102],
 [101, 1133, 1195, 1132, 1280, 102],
 [101, 1132, 1280, 1106, 3325, 102],
 [101, 1106, 3325, 1122, 4050, 102],
 [101, 1122, 4050, 119, 102]]

In [41]:
inputs

{'input_ids': [[101, 1188, 5650, 1110, 1136, 102], [101, 1110, 1136, 1315, 1263, 102], [101, 1315, 1263, 1133, 1195, 102], [101, 1133, 1195, 1132, 1280, 102], [101, 1132, 1280, 1106, 3325, 102], [101, 1106, 3325, 1122, 4050, 102], [101, 1122, 4050, 119, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 0, 0, 0, 0, 0, 0]}

In [42]:
print(inputs.keys())

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])


In [43]:
print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0]


In [44]:
sentences = [
    "This sentence is not too long but we are going to split it anyway.",
    "This sentence is shorter but will still get split.",
]
inputs = tokenizer(
    sentences, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [45]:
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

In [46]:
_ = inputs.pop("overflow_to_sample_mapping")
offsets = inputs.pop("offset_mapping")

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

torch.Size([2, 384])


In [47]:
outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([2, 384]) torch.Size([2, 384])


In [48]:
sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
# Mask all the [PAD] tokens
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))

start_logits[mask] = -10000
end_logits[mask] = -10000

In [49]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [50]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores).argmax().item()

    start_idx = idx // scores.shape[1]
    end_idx = idx % scores.shape[1]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

[(0, 18, 0.3386707305908203), (173, 184, 0.9714868664741516)]


In [51]:
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)

{'answer': '\n🤗 Transformers: State of the Art NLP', 'start': 0, 'end': 37, 'score': 0.3386707305908203}
{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1892, 'end': 1919, 'score': 0.9714868664741516}
